# Example

In [ ]:
%load_ext autoreload
%autoreload 2

%env GS_BACKEND=native_add3
%env CUDA_VISIBLE_DEVICES=0

from omegaconf import OmegaConf

### [0] Load images

In [ ]:
WS = "./demo/duck"
PRE_CFG_FN = "./profile/demo/demo_prep.yaml"
FIT_CFG_FN = "./profile/demo/demo_fit.yaml"

In [ ]:
from mosca_precompute import load_imgs_from_dir

img_list, img_fns = load_imgs_from_dir(WS)
pre_cfg = OmegaConf.load(PRE_CFG_FN)
# pre_cfg.tap_chunk_size = 4096 # reduce this for smaller gpu

### [1] Precompute Priors

In [ ]:
from mosca_precompute import get_moca_processor, preprocess

moca_processor = get_moca_processor(pre_cfg)

In [ ]:
preprocess(
    img_list=img_list,
    img_fns=img_fns,
    ws=WS,
    moca_processor=moca_processor,
    pre_cfg=pre_cfg,
)

### [2] Recon Stage


In [ ]:
from mosca_reconstruct import setup_recon_ws

fit_cfg = OmegaConf.load(FIT_CFG_FN)

logdir = setup_recon_ws(WS, fit_cfg=fit_cfg)
print(logdir)

In [ ]:
from lite_moca_reconstruct import static_reconstruct

static_reconstruct(WS, logdir, fit_cfg)

In [ ]:
from mosca_reconstruct import photometric_warmup

photometric_warmup(WS, logdir, fit_cfg) # this is optional, will auto skip by default.

In [ ]:
from mosca_reconstruct import scaffold_reconstruct

scaffold_reconstruct(WS, logdir, fit_cfg)

In [ ]:
from mosca_reconstruct import photometric_reconstruct

photometric_reconstruct(WS, logdir, fit_cfg)

### [3] viz and fps

In [ ]:
from mosca_evaluate import test_fps

test_fps(logdir)

In [ ]:
from mosca_viz import viz_main
import os.path as osp

viz_main(osp.join(logdir, "viz"), logdir, FIT_CFG_FN, N=5, back_ratio_3d=1.5)